# Working with GroupBy

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.DataFrame({"key":list("ABC")*2,
                 "data1":range(6),
                 "data2":np.arange(5,11)})

In [3]:
df

,key,data1,data2
0,A,0,5
1,B,1,6
2,C,2,7
3,A,3,8
4,B,4,9
5,C,5,10


In [4]:
group=df.groupby("key")

In [5]:
group.aggregate(["min",np.median,"max"])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     5    6.5   8
B       1    2.5   4     6    7.5   9
C       2    3.5   5     7    8.5  10

In [6]:
group.agg({"data1":"min","data2":"max"})

,data1,data2
key,,
A,0,8
B,1,9
C,2,10


In [7]:
def f(x):
    return x.max()-x.min()

In [8]:
group.agg(f)

,data1,data2
key,,
A,3,3
B,3,3
C,3,3


## Applying more than one function

In [9]:
data=pd.DataFrame({"letter":list("ABC")*4,
                   "num":["one","two"]*6,
                   "d1":np.random.randn(12),
                   "d2":np.arange(10,33,2)})

In [10]:
data

,letter,num,d1,d2
0,A,one,-1.239231,10
1,B,two,1.260503,12
2,C,one,0.824031,14
3,A,two,-1.679503,16
4,B,one,2.512805,18
5,C,two,0.683827,20
6,A,one,1.446127,22
7,B,two,1.235868,24
8,C,one,0.443357,26
9,A,two,0.523944,28


In [11]:
group=data.groupby(["letter","num"])

In [12]:
group_d1=group["d1"]

In [13]:
group_d1.agg("mean")

letter  num
A       one    0.103448
        two   -0.577780
B       one    1.418136
        two    1.248186
C       one    0.633694
        two    0.979782
Name: d1, dtype: float64

In [14]:
group_d1.agg(["mean","std",f])

mean       std         f
letter num                              
A      one  0.103448  1.898835  2.685358
       two -0.577780  1.558072  2.203447
B      one  1.418136  1.548095  2.189337
       two  1.248186  0.017419  0.024635
C      one  0.633694  0.269178  0.380674
       two  0.979782  0.418543  0.591910

In [15]:
group_d1.agg([("f_mean","mean"),
              ("f_std",np.std)])

f_mean     f_std
letter num                    
A      one  0.103448  1.898835
       two -0.577780  1.558072
B      one  1.418136  1.548095
       two  1.248186  0.017419
C      one  0.633694  0.269178
       two  0.979782  0.418543

In [16]:
group.agg({"d1":["count","max","mean"],
           "d2":"sum"})

d1                      d2
           count       max      mean sum
letter num                              
A      one     2  1.446127  0.103448  32
       two     2  0.523944 -0.577780  44
B      one     2  2.512805  1.418136  48
       two     2  1.260503  1.248186  36
C      one     2  0.824031  0.633694  40
       two     2  1.275737  0.979782  52

In [17]:
data.groupby(["letter","num"],
             as_index=False).mean()

,letter,num,d1,d2
0,A,one,0.103448,16.0
1,A,two,-0.577780,22.0
2,B,one,1.418136,24.0
3,B,two,1.248186,18.0
4,C,one,0.633694,20.0
5,C,two,0.979782,26.0


## Split-Apply-Combine

In [18]:
data

,letter,num,d1,d2
0,A,one,-1.239231,10
1,B,two,1.260503,12
2,C,one,0.824031,14
3,A,two,-1.679503,16
4,B,one,2.512805,18
5,C,two,0.683827,20
6,A,one,1.446127,22
7,B,two,1.235868,24
8,C,one,0.443357,26
9,A,two,0.523944,28


In [19]:
group=data.groupby("letter")

In [20]:
group["d2"].apply(lambda x:x.describe())

letter       
A       count     4.000000
        mean     19.000000
        std       7.745967
        min      10.000000
        25%      14.500000
        50%      19.000000
        75%      23.500000
        max      28.000000
B       count     4.000000
        mean     21.000000
        std       7.745967
        min      12.000000
        25%      16.500000
        50%      21.000000
        75%      25.500000
        max      30.000000
C       count     4.000000
        mean     23.000000
        std       7.745967
        min      14.000000
        25%      18.500000
        50%      23.000000
        75%      27.500000
        max      32.000000
Name: d2, dtype: float64

In [21]:
math=pd.DataFrame({"Class":list("AB")*3,
                   "Stu":["Kim","Sam",
                          "Tim","Tom","John","Kate"],
                   "Score":[60,70,np.nan,
                            55,np.nan,80]})
math

,Class,Stu,Score
0,A,Kim,60.0
1,B,Sam,70.0
2,A,Tim,NaN
3,B,Tom,55.0
4,A,John,NaN
5,B,Kate,80.0


In [22]:
group=math.groupby("Class")

In [23]:
group.mean()

,Score
Class,
A,60.000000
B,68.333333


In [24]:
func=lambda f:f.fillna(f.mean())

In [25]:
group.apply(func)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1160\2055239539.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  func=lambda f:f.fillna(f.mean())


,Class,Stu,Score
0,A,Kim,60.0
1,B,Sam,70.0
2,A,Tim,60.0
3,B,Tom,55.0
4,A,John,60.0
5,B,Kate,80.0


In [26]:
value={"A":100,"B":50}

In [27]:
func1=lambda f:f.fillna(value[f.name])

In [28]:
group.apply(func1)

,Class,Stu,Score
0,A,Kim,60.0
1,B,Sam,70.0
2,A,Tim,100.0
3,B,Tom,55.0
4,A,John,100.0
5,B,Kate,80.0
